# Repositorio PUCP Posgrado

In [43]:
###-Repositorio estadistica informatica pregrado UNALM-### 

#Trabajo Final → AVANCE.
#1. Relación de repositorios con sus respectivos url
#2. Automatización de la descarga de la información de un repositorio.
#3. Estructuración de la información descargada en formato csv

url = 'https://tesis.pucp.edu.pe/repositorio/handle/20.500.12404/757/recent-submissions'

#paquetes 
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 
import pandas as pd

import ssl
context = ssl._create_unverified_context() #la pagina pide un socket seguro 

#Las tesis se encuentran en 4 paginas distinas 

#devuelve la url de la siguiente pagina 
def next_page(url):
    page = urllib.request.urlopen(url,context = context)
    soup = BeautifulSoup(page, "html.parser")
    
    line = soup.find('a',{'class':'next-page-link'})
    line = line.decode().strip()
    offset = re.findall(r'\?offset=.{2}',line)
    url_diff = offset[0]
    
    url = re.sub('\?offset.*','',url)
    n_url = url + url_diff
    return n_url

URL = []
url1 = url

#devuelve las url de todas las paginas 
def all_pages(url,URL):
    
    page = urllib.request.urlopen(url,context = context)
    soup = BeautifulSoup(page, "html.parser")
    last = soup.find('li',{'class':'next pull-right disabled'})
    URL.append(url)
    if last == None: 
        url = next_page(url)
        all_pages(url,URL)
    else: 
        return URL 
    return URL 
all_pages(url1,URL)

###Almacenar links de las tesis 

handles = [] 

#las url de las tesis se diferencian por el codigo handle/20.500.12404/***** <-- diferencia 
for urls in URL: 
    page = urllib.request.urlopen(urls,context = context)
    soup = BeautifulSoup(page, "html.parser")
    for lines in soup.find_all('h4'):
        lines = lines.decode().strip()
        link = re.findall(r'(?<=20.500.12404/).+?(?=\")',lines) #handle unico a cada tesis (#12996)
        handles.append(link)
       
## Links completos 

#creando las url especificas de cada tesis 
links=[]
def get_links(handles,links):
    handle_url = re.sub('(?<=20.500.12404/).+','',url)
    for i in range(len(handles)): 
        links.append(handle_url+handles[i][0])
    return links

get_links(handles,links)    

#agregando ?show=full a las url se muestra informacion mas detallada de cada tesis 
links_full = [] 
for link in links: 
    links_full.append(link+'?show=full')
    

## almacenando en diccionario 

rep_PUCP = { 
        'unis' : [],
        'titulos' : [],
        'nombres' : [],
        'grados' : [], 
        'asesores' : [],
        'temas':[],
        'resumenes': [],
        'años' : []}
 
def get_all(links_full,reps_PUCP):
    for i in range(len(links_full)): 
        page = urllib.request.urlopen(links_full[i],context = context)
        soup = BeautifulSoup(page, "html.parser")
        full_t=''
        for line in soup.find_all('tr'):
            full_t = line.decode().strip()+'\n'+full_t
        
        uni = re.findall(r'(?<=>thesis\.degree\.grantor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        titulo = re.findall(r'(?<=>dc\.title</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        nombre = re.findall(r'(?<=>dc\.contributor\.author</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        grado = re.findall(r'(?<=>thesis\.degree\.name</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        asesor = re.findall(r'(?<=>dc\.contributor\.advisor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        resumen = re.findall(r'(?<=<tr class=\"ds-table-row even\">\n<td class=\"label-cell\">dc\.description\.abstract</td><td class=\"word-break\">)[\s\S]*?(?=</td><td>)',full_t)
        año = re.findall(r'(?<=>dc\.date\.issued</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        
        temas_prompt = r'(?<=>dc.subject</td><td class=\"word-break\">).+?(?=</td>)'
        temas_match = re.findall(temas_prompt,full_t)
                
        rep_PUCP['unis'].append(uni[0])
        rep_PUCP['titulos'].append(titulo[0])     
        rep_PUCP['nombres'].append(nombre[0])
        rep_PUCP['grados'].append(grado[0]) 
        
        if asesor == []: 
            rep_PUCP['asesores'].append('Sin asesor')
        else:
            rep_PUCP['asesores'].append(asesor[0])
        
        tema = ''
        for part in temas_match: #findall devuelve una lista 
            if re.findall(r'--',part) != None:
                part = re.sub(r'--.*','',part)
                tema = part + ','+ tema 
            else :
                tema = part + ','+ tema 
        rep_PUCP['temas'].append(tema)
        
        if resumen == []: 
            rep_PUCP['resumenes'].append('Sin resumen')
        else:
            rep_PUCP['resumenes'].append(resumen[0])
        rep_PUCP['años'].append(año[0])
    return reps_PUCP

PUCP = get_all(links_full,rep_PUCP)

In [45]:
df_PUCP = pd.DataFrame(PUCP)
df_PUCP

,unis,titulos,nombres,grados,asesores,temas,resumenes,años
0,Pontificia Universidad Católica del Perú. Escu...,Estimación del ciclo financiero utilizando mét...,"Atoche Murrieta, Lilian Katherine",Magíster en Estadística,"Camiz, Sergio","Análisis financiero,Sistema financiero,Indicad...",La presente tesis tiene por objetivo identific...,2022-09-22
1,Pontificia Universidad Católica del Perú. Escu...,Método para la fusión de categorías usando téc...,"Farro Diaz, Victor Daniel",Magíster en Estadística,"Bayes Rodriguez, Cristian Luis","Modelos lineales (Estadística),Estadística bay...","En la actualidad, muchas organizaciones dispon...",2022-04-28
2,Pontificia Universidad Católica del Perú. Escu...,Redes neuronales convolucionales para datos co...,"Cotacallapa Amanqui, Pavel Arturo",Magíster en Estadística,"Benites Sanchez, Luis Enrique","Redes neuronales,Procesamiento de datos,Variab...",En muchas situaciones prácticas es necesario e...,2022-04-07
3,Pontificia Universidad Católica del Perú. Escu...,Modelo de regresión cuantílica para respuestas...,"Manrique Urbina, Justo Andrés",Magíster en Estadística,"Bayes Rodriguez, Cristian Luis","Análisis de regresión,Inferencia estadística,",La presente tesis propone un modelo de regresi...,2022-03-21
4,Pontificia Universidad Católica del Perú. Escu...,Clasiﬁcación de riesgo para frecuencias y seve...,"Hernández Bello, Diana Patricia",Magíster en Estadística,"Valdivieso Serrano, Luis Hilmar","Tarifas,Seguros de automóviles,Estadística mat...","En la tariﬁcación de seguros generales, en par...",2022-03-16
...,...,...,...,...,...,...,...,...
56,Pontificia Universidad Católica del Perú. Escu...,Modelos de regresión binaria Skew probit para ...,"Pantoja Marin, Luis",Magíster en Estadística,"Bazán Guzmán, Jorge Luis","Estadística bayesiana,Estadística,Modelos mate...",La presente investigación se fundamenta en el ...,2013-02-05
57,Pontificia Universidad Católica del Perú. Escu...,Modelo de Rasch dicotómico con aplicación a la...,"Chincaro Del Coral, Omar Antonio",Magíster en Estadística,"Bazán Guzmán, Jorge Luis","Estadística bayesiana,Educación - Estadística,",En investigaciones de origen cuantitativo gene...,2012-08-17
58,Pontificia Universidad Católica del Perú. Escu...,Estimación no paramétrica en un proceso de Mar...,"Requena Espinoza, Genaro",Magíster en Estadística,"Doig Camino, Elizabeth","Estadística no paramétrica,Modelos de Markov,A...","En el presente trabajo, se estudian las propie...",2012-08-16
59,Pontificia Universidad Católica del Perú. Escu...,Modelos testlet logísticos y logísticos de exp...,"Flores Ari, Sandra Elizabeth",Magíster en Estadística,"Bazán Guzmán, Jorge Luis","Estadística bayesiana,Distribución (Teoría de ...",Los modelos de Teoría de Respuesta al Item (TR...,2012-08-16


# Repositorio UNALM Pregrado

In [7]:
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 
import pandas as pd

url = 'https://repositorio.lamolina.edu.pe/handle/20.500.12996/26/recent-submissions'

# url de la siguiente pagina
def next_page(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    
    line = soup.find('a',{'class':'next-page-link'})
    line = line.decode().strip()
    offset = re.findall(r'\?offset=.{2}',line)
    url_diff = offset[0]
    
    url = re.sub('\?offset.*','',url)
    n_url = url + url_diff
    return n_url

#Se extrae todos los url de todas las paginas 
URL = []
url1 = url

def all_pages(url,URL):
    
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    last = soup.find('li',{'class':'next pull-right disabled'})
    URL.append(url)
    if last == None: 
        url = next_page(url)
        all_pages(url,URL)
    else: 
        return URL 
    return URL 
all_pages(url1,URL)

###Almacenar links de las tesis 

handles = [] 
#Se extraen los urls de todas las tesis 
for urls in URL: 
    page = urllib.request.urlopen(urls)
    soup = BeautifulSoup(page, "html.parser")
    for lines in soup.find_all('h4'):
        lines = lines.decode().strip()
        link = re.findall(r'(?<=20.500.12996/).+?(?=\")',lines) #handle unico a cada tesis 
        handles.append(link)

## Links completos 

links=[]
def get_links(handles,links):
    handle_url = re.sub('(?<=20.500.12996/).+','',url)
    for i in range(len(handles)): 
        links.append(handle_url+handles[i][0])
    return links

get_links(handles,links)    

links_full = [] 
for link in links: 
    links_full.append(link+'?show=full')

# Dicionario de repositorio UNALM
rep_unalm = { 
        'unis' : [],
        'titulos' : [],
        'nombres' : [],
        'grados' : [], 
        'asesores' : [],
        'temas' : [],
        'resumenes': [],
        'años' : [] }
        
# Se define una función para obtener el nombre de la institución, el título de la tesis, el nombre del tesista, el grado al que aspira, su asesor, el resumen de la tesis y el año en que se publicó.
def get_all(links_full,reps_unalm):
    for i in range(len(links_full)): 
        page = urllib.request.urlopen(links_full[i])
        soup = BeautifulSoup(page, "html.parser")
        full_t=''
        for line in soup.find_all('tr'):
            full_t = line.decode().strip()+'\n'+full_t
        #se hace uso de regex para encontrar todo lo pedido    
        uni = re.findall(r'(?<=>thesis\.degree\.grantor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        titulo = re.findall(r'(?<=>dc\.title</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        nombre = re.findall(r'(?<=>dc\.contributor\.author</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        grado = re.findall(r'(?<=>thesis\.degree\.name</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        asesor = re.findall(r'(?<=>dc\.contributor\.advisor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        resumen = re.findall(r'(?<=>dc\.description\.abstract</td><td class=\"word-break\">).+?(?=</td><td>es_PE</td>)',full_t)
        año = re.findall(r'(?<=>dc\.date\.accessioned</td><td class=\"word-break\">).+?(?=T)',full_t)
        
        temas_prompt = r'(?<=>dc.subject</td><td class=\"word-break\">).+?(?=</td>)'
        temas_match = re.findall(temas_prompt,full_t)
        
        rep_unalm['unis'].append(uni[0])
        rep_unalm['titulos'].append(titulo[0])     
        rep_unalm['nombres'].append(nombre[0])
        rep_unalm['grados'].append(grado[0]) 
        
        if asesor == []: 
            rep_unalm['asesores'].append('N/A')
        else:
            rep_unalm['asesores'].append(asesor[0])
        
        tema = ''
        for part in temas_match: #findall devuelve una lista 
            tema = part + ','+ tema 
        rep_unalm['temas'].append(tema)
        
        rep_unalm['resumenes'].append(resumen[0])
        rep_unalm['años'].append(año[0])
    return reps_unalm

UNALM = get_all(links_full,rep_unalm)

In [47]:
df_UNALM = pd.DataFrame(UNALM)
df_UNALM


,unis,titulos,nombres,grados,asesores,temas,resumenes,años
0,Universidad Nacional Agraria La Molina. Facult...,Predicción del siniestro de vehículos particul...,"Ramirez Navarro, Vanessa Judith",Ingeniero Estadístico Informático,"Chue Gallardo, Jorge","Vehículos,Siniestros,Accidentes,Seguros,Estrat...",Tras el incremento de siniestros vehiculares d...,2022-12-30
1,Universidad Nacional Agraria La Molina. Facult...,Identificación de clientes en campañas para un...,"Alvarez Chancasanampa, Julio César",Ingeniero Estadístico Informático,"Miranda Villagómez, Clodomiro Fernando","Método Stacking,Desarrollo económico,Instituci...","La presente investigación, tiene como objetivo...",2022-11-04
2,Universidad Nacional Agraria La Molina. Facult...,Implementación de una solución de business int...,"Tinco Curi, Elizabeth Irene",Ingeniero Estadístico Informático,"Soto Rodríguez, Iván Dennys","Hospitales,Toma de decisiones,Soporte de decis...",Actualmente el sector salud dispone de una gra...,2022-03-29
3,Universidad Nacional Agraria La Molina. Facult...,Identificación de clientes que realizaron fuga...,"Marquez Meza, Francisco",Ingeniero Estadístico Informático,"Chue Gallardo, Jorge","Telecomunicaciones,Empresas,Consumidores,Compo...",El presente trabajo tiene como objetivo ilustr...,2022-03-29
4,Universidad Nacional Agraria La Molina. Facult...,Estimación del monto de siniestros ocurridos y...,"Alarcón Pimentel, Sandra Elena",Ingeniero Estadístico e Informático,"Menacho Chiok, César Higinio","Double Chain Ladder,Siniestros,Análisis de cos...",El sector asegurador peruano es supervisado en...,2022-02-17
...,...,...,...,...,...,...,...,...
58,Universidad Nacional Agraria La Molina. Facult...,Perfil de los clientes que aceptan una tarjeta...,"Acosta Pizarro, Diana Rosa",Ingeniero Estadístico e Informático,N/A,"Algoritmo Chaid exhaustivo,Bancos,Crédito,Cons...",El presente estudio tiene como objetivo princi...,2017-01-13
59,Universidad Nacional Agraria La Molina. Facult...,Identificación de un modelo explicativo de ret...,"Huamaní Miranda, María Alejandra",Ingeniero Estadístico e Informático,N/A,"Arboles de clasificación Cart,Bancos,Consumido...",La realidad competitiva que en estos días enfr...,2017-01-13
60,Universidad Nacional Agraria La Molina. Facult...,Uso del criterio AHP para la toma de decisiones,"Loaiza Alamo, Marco Antonio",Ingeniero Estadístico e Informático,"Sotomayor Ruíz, Rino Nicanor","Qlikview,Microsoft project,Eviews,SQL,SPSS,Min...",El objetivo de esta investigación es determina...,2017-01-10
61,Universidad Nacional Agraria La Molina. Facult...,Muestreo de unidades agrícolas a través de pun...,"Alarcón Novoa, Jorge Alfonso",Ingeniero Estadístico,"Rubio Donet, Arturo","Muestreo,Análisis estadístico,Agricultura,Perú,",En esta investigación se presenta un método de...,2016-08-17


# Repositorio UNALM Posgrado

In [39]:
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 
import pandas as pd

url = 'http://repositorio.lamolina.edu.pe/handle/20.500.12996/329'

# Se importa las librerias necesarias
import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 


#Se extrae todos los url de todas las paginas 
URL = []

def all_pages(url,URL):
    
    page = urllib.request.urlopen(url,context=context)
    soup = BeautifulSoup(page, "html.parser")
    last = soup.find('li',{'class':'next pull-right disabled'})
    URL.append(url)
    return URL 
all_pages(url,URL)

###Almacenar links de las tesis 

handles = [] 
#Se ectraen los urls de todas las tesis 
for urls in URL: 
    page = urllib.request.urlopen(urls,context = context)
    soup = BeautifulSoup(page, "html.parser")
    for lines in soup.find_all('h4'):
        lines = lines.decode().strip()
        link = re.findall(r'(?<=20.500.12996/).+?(?=\")',lines) #handle unico a cada tesis 
        handles.append(link)

## Links completos 

links=[]
def get_links(handles,links):
    handle_url = re.sub('(?<=20.500.12996/).+','',url)
    for i in range(len(handles)): 
        links.append(handle_url+handles[i][0])
    return links

get_links(handles,links)    

links_full = [] 
for link in links: 
    links_full.append(link+'?show=full')
  

# Dicionario de repositorio UNALM
rep_unalmp = { 
        'unis' : [],
        'titulos' : [],
        'nombres' : [],
        'grados' : [], 
        'asesores' : [],
        'temas':[],
        'resumenes': [],
        'años' : [] }
        
# Se define una función para obtener el nombre de la institución, el título de la tesis, el nombre del tesista, el grado al que aspira, su asesor, el resumen de la tesis y el año en que se publicó.
def get_all(links_full,reps_unalmp):
    for i in range(len(links_full)): 
        page = urllib.request.urlopen(links_full[i],context = context)
        soup = BeautifulSoup(page, "html.parser")
        full_t=''
        for line in soup.find_all('tr'):
            full_t = line.decode().strip()+'\n'+full_t
        #se hace uso de regex para encontrar todo lo pedido    
        uni = re.findall(r'(?<=>thesis\.degree\.grantor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        titulo = re.findall(r'(?<=>dc\.title</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        nombre = re.findall(r'(?<=>dc\.contributor\.author</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        grado = re.findall(r'(?<=>thesis\.degree\.name</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        asesor = re.findall(r'(?<=>dc\.contributor\.advisor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
        resumen = re.findall(r'(?<=>dc\.description\.abstract</td><td class=\"word-break\">).+?(?=</td><td>es_PE</td>)',full_t)
        año = re.findall(r'(?<=>dc\.date\.accessioned</td><td class=\"word-break\">).+?(?=T)',full_t)
        
        temas_prompt = r'(?<=>dc.subject</td><td class=\"word-break\">).+?(?=</td>)'
        temas_match = re.findall(temas_prompt,full_t)
        
        if uni == []:
            rep_unalmp['unis'].append('N/A')
        else: 
            rep_unalmp['unis'].append(uni)
        if titulo == []:
            rep_unalmp['titulos'].append('N/A')
        else: 
            rep_unalmp['titulos'].append(titulo[0])
        rep_unalmp['nombres'].append(nombre[0])
        rep_unalmp['grados'].append(grado[0]) 
        
        if asesor == []: 
            rep_unalmp['asesores'].append('N/A')
        else:
            rep_unalmp['asesores'].append(asesor[0])
        tema = ''
        for part in temas_match: #findall devuelve una lista 
            tema = part + ','+ tema 
        rep_unalmp['temas'].append(tema)
        
        rep_unalmp['resumenes'].append(resumen[0])
        rep_unalmp['años'].append(año[0])
    return reps_unalmp

UNALMp = get_all(links_full,rep_unalmp)

In [46]:
df_UNALMp = pd.DataFrame(UNALMp)
df_UNALMp


,unis,titulos,nombres,grados,asesores,temas,resumenes,años
0,[Universidad Nacional Agraria La Molina. Escue...,Clasificación de la eficiencia del gasto públi...,"Romero Cuadros, Italo Brayan",Magister Scientiae - Estadística Aplicada,"Gamboa Unsihhuay, Jesús Eduardo","Gobierno regional,Gastos públicos,Indicadores ...",Esta investigación tiene como objetivo de dete...,2022-10-04
1,[Universidad Nacional Agraria La Molina. Escue...,Factores que determinan la calidad de servicio...,"Chumpitaz Ramos, Domingo Guzmán",Magister Scientiae - Estadística Aplicada,"Sotomayor Ruíz, Rino Nicanor","Servicio al cliente,Calidad,Satisfacción del a...","Los Modelos de Ecuaciones Estructurales (SEM),...",2021-04-05
2,[Universidad Nacional Agraria La Molina. Escue...,Influencia de la violencia contra las mujeres ...,"Asencios Gonzalez, Zaida Beatriz",Magister Scientiae - Estadística Aplicada,"Porras Cerrón, Jaime Carlos","Mujeres,Perú,Violencia familiar,Calidad de vid...","En la presente investigacion, se empleo Modelo...",2019-12-30
3,[Universidad Nacional Agraria La Molina. Escue...,Aplicación de los modelos de ecuaciones estruc...,"Chafloque Cespedes, María Raquel",Magister Scientiae - Estadística Aplicada,"Miranda Villagomez, Clodomiro Fernando","Modelos de ecuaciones,",Los Modelos de Ecuaciones Estructurales con Mí...,2019-12-23
4,[Universidad Nacional Agraria La Molina. Escue...,Comparación del pronóstico de riesgo de crédit...,"Valdivia Carbajal, Manuel",Magister Scientiae - Estadística Aplicada,"Chue Gallardo, Jorge","Estadística,",Esta tesis toma como caso de estudio a una emp...,2019-11-25
5,[Universidad Nacional Agraria La Molina. Escue...,Equiparaciòn de puntuaciones en el examen de a...,"Rado Huaringa, Joao Manuel",Magister Scientiae - Estadística Aplicada,"Porras Cerrón, Jaime Carlos","Estudiantes,Universidades,Métodos estadísticos...",En esta investigación se realizó la aplicación...,2019-09-24
6,[Universidad Nacional Agraria La Molina. Escue...,Modelo de ecuación estructural explicativo del...,"Salazar Vega, Rolando Jesús",Magister Scientiae - Estadística Aplicada,"Rosas Villena, Fernando René","Estudiantes,Docentes,Estadísticas,Universidade...",El propósito principal de la investigación fue...,2019-07-24
7,[Universidad Nacional Agraria La Molina. Escue...,Uso de los modelos heterocedásticos con Bootst...,"Orosco Gavilán, Juan Carlos",Magister Scientiae - Estadística Aplicada,"Febres Huamán, Grimaldo","Bolsa de Valores de Lima,Análisis de series cr...",La presente investigación es de naturaleza apl...,2019-04-12
8,[Universidad Nacional Agraria La Molina. Escue...,Clasificación de especies forestales maderable...,"Montenegro Muro, Rolando Antonio",Magister Scientiae - Estadística Aplicada,"Acevedo Mallque, Moisés Pascual","Arboles forestales,Amazonía,Clasificación,Vari...",El bosque amazónico cuenta con una gran varied...,2018-12-27
9,[Universidad Nacional Agraria La Molina. Escue...,Detección de datos multivariados atípicos con ...,"Rubio Donet, Jorge Luis",Magister Scientiae - Estadística Aplicada,"Miranda Villagomez, Clodomiro Fernando","Análisis de series cronológicas,Métodos estadí...",La presencia de observaciones atípicas en un c...,2018-11-06


In [49]:
#se une todas las informaciones obtenidas
Tesis = pd.concat([df_PUCP, df_UNALM, df_UNALMp], axis=0)
#Tesis
Tesis.to_csv('Tesis.csv', index=False,encoding='utf-8-sig',
          header = ['Institución','Título de la tesis', 'Nombre del tesista', 'Grado', 'Nombre del asesor','Tema', 'Resumen','Año'])
df_Tesis_read = pd.read_csv('Tesis.csv')
df_Tesis_read


,Institución,Título de la tesis,Nombre del tesista,Grado,Nombre del asesor,Tema,Resumen,Año
0,Pontificia Universidad Católica del Perú. Escu...,Estimación del ciclo financiero utilizando mét...,"Atoche Murrieta, Lilian Katherine",Magíster en Estadística,"Camiz, Sergio","Análisis financiero,Sistema financiero,Indicad...",La presente tesis tiene por objetivo identific...,2022-09-22
1,Pontificia Universidad Católica del Perú. Escu...,Método para la fusión de categorías usando téc...,"Farro Diaz, Victor Daniel",Magíster en Estadística,"Bayes Rodriguez, Cristian Luis","Modelos lineales (Estadística),Estadística bay...","En la actualidad, muchas organizaciones dispon...",2022-04-28
2,Pontificia Universidad Católica del Perú. Escu...,Redes neuronales convolucionales para datos co...,"Cotacallapa Amanqui, Pavel Arturo",Magíster en Estadística,"Benites Sanchez, Luis Enrique","Redes neuronales,Procesamiento de datos,Variab...",En muchas situaciones prácticas es necesario e...,2022-04-07
3,Pontificia Universidad Católica del Perú. Escu...,Modelo de regresión cuantílica para respuestas...,"Manrique Urbina, Justo Andrés",Magíster en Estadística,"Bayes Rodriguez, Cristian Luis","Análisis de regresión,Inferencia estadística,",La presente tesis propone un modelo de regresi...,2022-03-21
4,Pontificia Universidad Católica del Perú. Escu...,Clasiﬁcación de riesgo para frecuencias y seve...,"Hernández Bello, Diana Patricia",Magíster en Estadística,"Valdivieso Serrano, Luis Hilmar","Tarifas,Seguros de automóviles,Estadística mat...","En la tariﬁcación de seguros generales, en par...",2022-03-16
...,...,...,...,...,...,...,...,...
139,['Universidad Nacional Agraria La Molina. Escu...,Regresión bayesiana con enlaces asimétricos pa...,"Fernández Vásquez, Richard Fernando",Magister Scientiae - Estadística Aplicada,"Chue Gallardo, Jorge","Bancos,Consumidores,Préstamos,Crédito comercia...",En la actualidad las entidades bancarias convi...,2018-02-20
140,['Universidad Nacional Agraria La Molina. Escu...,Estimación de componentes de varianza utilizan...,"Vargas Paredes, Ana Cecilia",Magister Scientiae - Estadística Aplicada,"Maehara Oyata, Víctor Manuel","Vacas lecheras,Razas (animales),Heredabilidad,...",Se estimó mediante un modelo lineal mixto los ...,2017-11-13
141,['Universidad Nacional Agraria La Molina. Escu...,Modelos semiparamétricos de eventos recurrente...,"Rebaza Fernández, Diana del Rocío",Magister Scientiae - Estadística Aplicada,"Maehara Oyata, Víctor Manuel","Glandulas mamarias,Neoplasmas,Mujeres,Métodos ...",La recurrencia de un evento en un paciente es ...,2017-11-13
142,['Universidad Nacional Agraria La Molina. Escu...,Modelos estadísticos en procesos puntuales esp...,"Quispe Quispe, Braulio",Magister Scientiae - Estadística Aplicada,"López de Castilla Vásquez, Carlos","Perú,Ciudades,Zonas urbanas,Seguridad,Modelos ...",La presente tesis plantea una aplicación de lo...,2017-10-03
